In [1]:
!nvidia-smi

Mon Aug 26 12:32:02 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        Off |   00000000:2F:00.0  On |                  Off |
|  0%   68C    P2            336W /  450W |   23374MiB /  24564MiB |     98%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [7]:
import numpy as np
import cv2
import torch
import os
import shutil

# Define top and bottom options with their respective color codes
top_colors = {
    "Sunny": (173, 216, 230),        # Light blue top
    "Rainy": (169, 169, 169),        # Gray top
    "Night": (139, 69, 19),             # Dark blue top
    "Cloudy Sunny": (192, 192, 224)  # Mixed white and blue top
}

bottom_colors = {
    "Road": (50, 50, 50),            # Black / gray bottom
    "Snowy": (255, 255, 255),        # White bottom
    "Rural": (0, 0, 64)           # Brown bottom
}

# Function to add noise to the image
def add_noise(image, noise_level=0.025):
    noise = np.random.normal(loc=0, scale=noise_level, size=image.shape)
    noisy_image = np.clip(image + noise * 255, 0, 255).astype(np.uint8)
    return noisy_image

# Function to resize the numpy image based on given options
def resize_numpy_image(image, max_resolution=512, resize_short_edge=True):
    h, w = image.shape[:2]
    if resize_short_edge:
        if h > w:
            scale = max_resolution / w
        else:
            scale = max_resolution / h
    else:
        scale = max_resolution / max(h, w)

    new_h, new_w = int(h * scale), int(w * scale)
    resized_image = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_CUBIC)
    resized_image = cv2.resize(resized_image, (new_w//64, new_h//64), interpolation=cv2.INTER_CUBIC)
    resized_image = cv2.resize(resized_image, (new_w, new_h), interpolation=cv2.INTER_NEAREST)
    return resized_image

# Function to convert image to tensor
def img2tensor(image):
    image = image.transpose(2, 0, 1)  # Convert to channel-first format
    tensor_image = torch.from_numpy(image).float() / 255.0
    return tensor_image

# Function to empty the directory
def empty_directory(directory):
    if os.path.exists(directory):
        shutil.rmtree(directory)  # Remove the directory and its contents
    os.makedirs(directory)  # Create the directory again

# Define the main function to generate templates
def generate_color_conditioning_templates():
    output_dir = "T2I-Adriver/dataset/color_templates"
    empty_directory(output_dir)  # Empty the directory before saving new images

    opt = type('', (), {})()  # Create a simple object to store options
    opt.max_resolution = 512
    opt.resize_short_edge = True

    for top_name, top_color in top_colors.items():
        for bottom_name, bottom_color in bottom_colors.items():
            # Create a base image
            image = np.zeros((512, 512, 3), dtype=np.uint8)

            # Set the top half to the top color and the bottom half to the bottom color
            image[:256, :] = top_color
            image[256:, :] = bottom_color

            # Add noise to the image
            noisy_image = add_noise(image)

            # Resize and process the image
            resized_image = resize_numpy_image(noisy_image, max_resolution=opt.max_resolution, resize_short_edge=opt.resize_short_edge)

            # Convert to tensor
            color_tensor = img2tensor(resized_image).unsqueeze(0)

            # Output the resulting tensor shape and save the image for verification
            print(f"{top_name}_{bottom_name} tensor shape: {color_tensor.shape}")
            
            # Save the image
            image_filename = f"{top_name}_{bottom_name}.png"
            cv2.imwrite(os.path.join(output_dir, image_filename), resized_image)

# Run the function to generate templates
generate_color_conditioning_templates()


Sunny_Road tensor shape: torch.Size([1, 3, 512, 512])
Sunny_Snowy tensor shape: torch.Size([1, 3, 512, 512])
Sunny_Rural tensor shape: torch.Size([1, 3, 512, 512])
Rainy_Road tensor shape: torch.Size([1, 3, 512, 512])
Rainy_Snowy tensor shape: torch.Size([1, 3, 512, 512])
Rainy_Rural tensor shape: torch.Size([1, 3, 512, 512])
Night_Road tensor shape: torch.Size([1, 3, 512, 512])
Night_Snowy tensor shape: torch.Size([1, 3, 512, 512])
Night_Rural tensor shape: torch.Size([1, 3, 512, 512])
Cloudy Sunny_Road tensor shape: torch.Size([1, 3, 512, 512])
Cloudy Sunny_Snowy tensor shape: torch.Size([1, 3, 512, 512])
Cloudy Sunny_Rural tensor shape: torch.Size([1, 3, 512, 512])
